In [ ]:
import sys
sys.setrecursionlimit(10**9) # 再帰関数の呼び出し制限
import pypyjit
pypyjit.set_param('max_unroll_recursion=-1') # pypyの再帰高速化

class RollbackUnionFind:
    """
    履歴を持ち、直前の操作を（順に）取り消せるUnionFind
    """

    def __init__(self, n: int):
        self.table = [-1] * n
        self.history = []
        # (継続リーダー番号1, 被統合リーダー番号2, 1統合前table値, 2統合前table値)
        # 無効なuniteの場合、被統合リーダー番号を"-1"とする

    def root(self, x: int):
        tbl = self.table
        while tbl[x] >= 0:
            x = tbl[x]
        return x
    
    def is_same(self, x: int, y: int):
        return self.root(x) == self.root(y)
    
    # グループの要素数が多い方にくっつける
    def unite(self, x: int, y: int):
        r1 = self.root(x)
        r2 = self.root(y)
        if r1 == r2:
            self.history.append((r1, -1, -1, -1))
            return False
        d1 = self.table[r1]
        d2 = self.table[r2]
        if d1 <= d2:
            self.table[r2] = r1
            self.table[r1] += d2
            self.history.append((r1, r2, d1, d2))
        else:
            self.table[r1] = r2
            self.table[r2] += d1
            self.history.append((r2, r1, d2, d1))
        return True
    
    def rollback(self):
        if not self.history:
            return False
        r1, r2, d1, d2 = self.history.pop()
        if r2 == -1:
            return False
        self.table[r1] = d1
        self.table[r2] = d2
        return True
    
    def get_size(self, x: int):
        return -self.table[self.root(x)]

def main():
    N,M,K = map(int,input().split())
    uf = RollbackUnionFind(N)
    uvw = []
    for _ in range(M):
        u,v,w = map(int,input().split())
        uvw.append((u-1,v-1,w))
    def dfs(cnt,idx,weight,uf,ans):
        if cnt == N-1:
            if ans > weight:
                ans = weight
            return ans
        for i in range(idx+1,M-N+cnt+2):
            u,v,w = uvw[i]
            if uf.is_same(u,v):
                continue
            uf.unite(u,v)
            ans = dfs(cnt+1,i,(weight+w)%K,uf,ans)
            uf.rollback()
        return ans
    print(dfs(0,-1,0,uf,float("inf")))

if __name__ == "__main__":
    main()